In [1]:
import os, sys
sys.path.append('./lung_cancer/')

# Training models for pulmonary nodule detection on LUNA16 dataset

In this Tutorial you will learn how to train models on popular [LUNA16](http://luna16.grand-challenge.org) competition dataset (scans from [LIDC-IDRI](https://wiki.cancerimagingarchive.net/display/Public/LIDC-IDRI).
You will learn how to:
1. Prepare dataset of CT-scans crops of smaller size
2. Train a ready-made model from recommended RadIO model zoo, e.g. [**Keras3DUnet**](https://analysiscenter.github.io/radio/api/keras_3dunet.html)
3. Train a popular model from dataset/models zoo, e.g. [**V-Net**](https://analysiscenter.github.io/dataset/api/dataset.models.tf.vnet.html)
4. Write your own model on TensorFlow and train it

### Part I. Prepare dataset of crops

Since it is not yet practically approachable to train a neural network on CT-scans, as they weight too much (300-500mb+), one approach which allows to tackle volumetric information would be to train on smaller 3D parts, `crops`. Cropping parts with/without nodules also allows for better sampling and balance of positive/negative examples in batch. RadIO contains ready-made preprocessing pipeline which will create a dataset with crops, you may use LUNA16 challenge dataset in MetaImage format for this tutorial.

In [2]:
from radio.pipelines import split_dump

You will need LUNA16 folder directory and `annotations.csv` provided for the competition.

In [5]:
import pandas as pd
nodules = pd.read_csv('/data/MRT/luna/CSVFILES/annotations.csv')
DIR_LUNA = '/data/MRT/luna/s*/*.mhd'

In [6]:
from radio.dataset import FilesIndex, Dataset
from radio import CTImagesMaskedBatch as CTIMB

lunaix = FilesIndex(path=DIR_LUNA, no_ext=True)
lunaset = Dataset(index=lunaix, batch_class=CTIMB)

Note, that you may also want to split your dataset to train/test/validation parts; It's possible by splitting indices of dataset instance by `.cv_split` method, for example, splitting 90% to train and 10% for test:

In [6]:
lunaset.cv_split(0.9, shuffle=True)

Now original dataset is split to train and test sub-datasets:

In [7]:
len(lunaset.train), len(lunaset.test)

(799, 89)

Next, we need to write down preprocessing workflow. 

**`split_dump()`** from `radio.pipelines` returns basic workflow, including normalizing Hounsfield Units, resizing all scans to fixed scans and in at same time unifying its spacing to desired one. After preprocessing `split_dump` will sample smaller crops of desired shape (32, 64, 64) with nodules and without. Workflow will also create masks based on nodules location and dump all the component, e.g. `masks`, `images`, into fast [`blosc`](http://blosc.org/pages/blosc-in-depth/) format.

Crops with nodules will be dumped to `cancer_path` and crops without any nodules will be dumped to `non_cancer_path` to allow managing balance of positive/negative examples in batch later during training.

You may want to pass additioanl paramteres as `kwargs`: 

In [8]:
SPACING = (1.7, 1.0, 1.0)  # spacing of scans after spacing unification
SHAPE = (400, 512, 512)  # shape of scans after spacing unification
PADDING = 'reflect'  # padding-mode that produces the least amount of artefacts
METHOD = 'pil-simd'  # robust resize-engine

kwargs_default = dict(shape=SHAPE, spacing=SPACING, padding=PADDING, method=METHOD)

In [9]:
crop_pipeline = split_dump(cancer_path='/data/lunaset_split/train/cancer/', 
                           non_cancer_path='/data/lunaset_split/train/ncancer/',
                           nodules=nodules, fmt='raw', nodule_shape=(32, 64, 64), batch_size=20, **kwargs_default)

Pass dataset directly to pipeline and run it, for example for `lunaset.train` sub-dataset:

(Note, it may take some time, so better grab a coffee)

In [ ]:
(lunaset.train >> crop_pipeline).run()

You may want to prepare test part in similar way, but do not forget to change `cancer_path` and `non_cancer_path`. In this Tutorial we will skip it.

### Part II. Train a ready made model in Keras: 3D U-Net

RadIO contains a small but growing zoo of models in Keras and Tensorflow. In this part you will perform training a 3D U-Net architecture for pulmonary nodule segmentation, training on crops you prepared earlier.

In [2]:
DIR_CANCER = '/data/lunaset_split_new/train/cancer/*'
DIR_NCANCER = '/data/lunaset_split_new/train/ncancer/*'

Similar to preprocessing, let's build a training pipeline. This time, you will need 2 datasets, one for crops with nodules and another with crops without them, that's why they were dumped in different folders. Remeber, that masks and some other special information is also dumped and will be easily loaded on request.

In [7]:
cix = FilesIndex(path=DIR_CANCER, dirs=True)
ncix = FilesIndex(path=DIR_NCANCER, dirs=True)

cancerset = Dataset(index=cix, batch_class=CTIMB)
ncancerset = Dataset(index=ncix, batch_class=CTIMB)

How can we use both datasets in a same time during training? Dataset allows to `merge` them specifying batch_size for each of them. Say, we would want take 4 crops with nodules and 4 without them, in total batch_size of 8 crops of (32, 64, 64) size. Let's import a ready made pipeline `combine_crops` for this problem and specify parameters:

In [8]:
from radio.pipelines import combine_crops

In [9]:
combine_pipeline = combine_crops(cancerset, ncancerset, batch_sizes=(4,4))

Next step, the model: import and define config in a dict. Need to specify input crops and masks sizes (will be same) as well as trainging model config: Optimizer, Loss function; e.g. use Adam for training and Dice loss funciton for segmentation:

In [15]:
from radio.models import Keras3DUNet
from radio.models.keras.losses import dice_loss

model_config = dict(
    input_shape = (1, 32, 64, 64),
    num_targets = 1,
    optimizer ='Adam',
    loss= dice_loss
)

Using TensorFlow backend.


After that, let's add one more part to training pipeline specifying model and training params by chaining `.init_model` and `.train_model` pipeline methods:

In [16]:
from radio.dataset import F

train_pipeline = (
    combine_pipeline
    .init_model('static', Keras3DUNet,name='3dunet',config=model_config)
    .train_model('3dunet',x=F(CTIMB.unpack, component='images',
                              data_format='channels_first'),
                          y= F(CTIMB.unpack, component='segmentation_targets',
                              data_format='channels_first'))
)

Here `train_model` employ custom `CTIMB.unpack` method that helps to unpack specific components from batch and put them into desired alias of keras network: `images` component goes to `x` and `segmentation_targets` put `masks` into `y`.

If you would want to try a keras network for classification, you could simply pass component for y's unpack to `classification_targets`. That way, unpack would transform masks into binary classification (`nodule` vs. `non-nodule`) target based on threshold of amount of cancer voxels inside the crop to label `nodule`. See [documentation](https://analysiscenter.github.io/radio/intro/models.html) for details.

Then, the moment of truth:

In [ ]:
train_pipeline.run()

After execution is finished, you may `.save` keras model:

In [19]:
unet = train_pipeline.get_model_by_name('3dunet')
unet.save('3dunet')

If you later would like to access it, that would be easy by adding `path` in `model_config` and redefining and running same pipeline again:

In [ ]:
model_config.update({'path': '3dunet'})

train_pipeline = (
    combine_pipeline
    .init_model('static', Keras3DUNet, name='3dunet',config=model_config)
    .train_model('3dunet',x=F(CTIMB.unpack, component='images',
                              data_format='channels_first'),
                          y=F(CTIMB.unpack, component='segmentation_targets',
                              data_format='channels_first'))
)

train_pipeline.run()

### Part 3. Training a model from radio.dataset.models zoo: V-Net

Dataset library used by RadIO contains a zoo of many popular model architecture in tensorflow. In this part you will learn to train V-Net (volumetric u-net) for same task with same preprocessing pipeline. All you need is to define config of dataset's v-net model:

In [12]:
from radio.dataset.models.tf import VNet
import tensorflow as tf

In [13]:
inputs_config = dict(
    images={'shape': (32, 64, 64, 1)},
    labels={'name': 'targets', 'shape': (32, 64, 64, 1)}
)


model_config = dict(
    inputs=inputs_config,
    optimizer='Adam',
    loss='sigmoid_cross_entropy',
)

model_config['input_block/inputs'] = 'images'
model_config['head/num_classes'] = 1
model_config['head/layout'] = 'cna'
model_config['head/kernel_size'] = 1
model_config['head/activation'] = tf.nn.sigmoid


Dataset.models has highly parametrized architecures which allows to perform experiments by combining different paramteres without changing model's code. It greatly facilitates clear and reproducible experiment settings.

In example above you are provided with V-Net model which will be trained on crops with Adam optimizer, sigmoid_cross_entropy loss from tensorflow for binary classification. Slight modifications are added to head of the network, last convolutional block would consist of convolution with (1, 1, 1) kernel, 1 filter, following batch normalisation layer and sigmoid activation function. See dataset [documentation](https://analysiscenter.github.io/dataset/intro/tf_models.html) for details.

In [ ]:
from radio.dataset import F

vnet_ppl = (
    combine_pipeline
    .init_model('static', VNet, 'vnet', config=model_config)
    .train_model('vnet',feed_dict={'images':  F(CTIMB.unpack, component='images',data_format='channels_last'),
                                   'targets': F(CTIMB.unpack, component='segmentation_targets')})
)

vnet_ppl.run()

### Part 4. Write you own tensorflow model and train it with same pipeline

In this part you will see, how to train your own models ith RadIO. The easiest way would be to write your model in tensorflow or keras and use `.import_model()` in pipeline. Let's build a 3D convolutional encoder-decoder. architecture.

The easiest way to be able to use `pipeline` would be to inherit `BaseModel` class and redefine its **`build`** method for building model's graph in tensorflow and also **`train`** method as it will be used by pipeline's `.train_model`. See documentation of [BaseModel](https://analysiscenter.github.io/dataset/api/dataset.models.html#dataset.models.BaseModel) to make sense of its possibilities.

In [1]:
import tensorflow as tf
from radio.dataset.models import BaseModel

In [2]:
class Custom_model(BaseModel):
    
    def __init__(self, *args, **kwargs):
        self.inputs = None
        self.targets = None
        self.output = None
        self.loss = None
        self.sess = None
        self.train_step = None
        self.graph = tf.Graph()
        super().__init__(*args, **kwargs)
    
    def build(self, *args, **kwargs):
        filters = [32, 64, 128, 256]
        with self.graph.as_default():
            with tf.variable_scope('custom', reuse=False):
                
                self.inputs = tf.placeholder(shape=(None, 32, 64, 64, 1), dtype=tf.float32)
                self.targets = tf.placeholder(shape=(None, 32, 64, 64, 1), dtype=tf.float32)
                
                x = self.inputs
                for f in filters:
                    x = tf.layers.conv3d(x, f, (3, 3, 3), padding='same', activation=tf.nn.relu)
                    x = tf.layers.conv3d(x, f, (3, 3, 3), padding='same', activation=tf.nn.relu)
                    x = tf.layers.max_pooling3d(x, pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')

                x = tf.layers.conv3d(x, f*2, (3, 3, 3), padding='same', activation=tf.nn.relu)
                x = tf.layers.conv3d(x, f*2, (3, 3, 3), padding='same', activation=tf.nn.relu)

                for f in filters[::-1]:
                    x = tf.layers.conv3d_transpose(x, f, (2, 2, 2), (2, 2, 2), use_bias=False, padding='same')
                    x = tf.layers.conv3d(x, f, (3, 3, 3), padding='same', activation=tf.nn.relu)
                    x = tf.layers.conv3d(x, f, (3, 3, 3), padding='same', activation=tf.nn.relu)


                self.output = tf.layers.conv3d(x, 1, (1, 1, 1), padding='same', activation=tf.nn.sigmoid)

                self.loss = tf.losses.sigmoid_cross_entropy(self.targets, self.output)
                self.train_step = tf.train.AdamOptimizer().minimize(self.loss)

            self.sess = tf.Session()
            self.sess.run(tf.global_variables_initializer())
            
    
    def train(self, x, y): 
        self.sess.run(self.train_step, feed_dict={self.inputs: x, self.targets: y})

Then you can use `.import_model` just like that, and not forget to provide right arguments (from train method) into `.train_model`.

In [ ]:
from radio.dataset import F

model = Custom_model()

cust_ppl = (
    combine_pipeline
    .import_model(name='custom', source=model)
    .train_model('custom', x = F(CTIMB.unpack, component='images',data_format='channels_last'),
                           y = F(CTIMB.unpack, component='segmentation_targets'))
)

cust_ppl.run()